<a href="https://colab.research.google.com/github/FranciscoBPereira/AnaliseDados2324-MEI-ISEC/blob/main/AD2324_P6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 5)


# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
#### Task 1 ####

# Using a pretrained neural network directly from Keras Applications to Classify Individual Images
# https://keras.io/api/applications/

# The ResNet-50 Model will be used to predict the category of selected images
# https://keras.io/api/applications/resnet/#resnet50-function

# Load Model with weights trained on Imagenet
modelR = tf.keras.applications.ResNet50(weights="imagenet")



In [ ]:
# Load two sample images and resize them to match the input required by ResNet

from sklearn.datasets import load_sample_images

images = load_sample_images()["images"]
images_resized = tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True)(images)


In [ ]:
# Print the images

plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.imshow(images_resized[0]/255.0)
plt.title('Palace')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Dahlia')
plt.imshow(images_resized[1]/255.0)
plt.axis('off')

plt.show()




In [ ]:
# Preprocess the sample images in a specific way that the model is expecting
# Call method: resnet.preprocess_input()

inputs = tf.keras.applications.resnet50.preprocess_input(images_resized)



In [ ]:
# Make prediction with the pretrained model

Y_proba = modelR.predict(inputs)

print('Shape: ', Y_proba.shape)

print('First Prediction: ', Y_proba[0])



In [ ]:
# Raw predictions are hard to understand
# Check the top predicted classes

top_K = tf.keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print(f"Image #{image_index}")
    for class_id, name, y_proba in top_K[image_index]:
        print(f"  {class_id} - {name:12s} {y_proba:.2%}")


Are the results correct? How do you interpret the outcomes?

In [ ]:
#### Task 2 ####

# Transfer Learning using Pretrained Neural Networks
# The same model ResNet-50 will be applied to the Flowers Dataset

# Data Fetching and Loading

import os
import pathlib

_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
path_to_zip = tf.keras.utils.get_file('flower_photos', origin=_URL, untar=True, cache_dir=os.curdir)
PATH = os.path.join(os.path.dirname(path_to_zip), 'flower_photos')
data_dir = pathlib.Path(PATH)

batch_size = 32


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  shuffle=True,
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  shuffle=True,
  batch_size=batch_size)

class_names = train_ds.class_names

In [ ]:
# Load ResNet50 from Keras Applications

ResNet_base = keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))

ResNet_base.trainable = False

**Quiz 1**

Explain the meaning of the following code snippets and justify why they are relevant for this transfer learning task:


1.   include_top = False
2.   ResNet_base.trainable = False


In [ ]:

# Create the complete model
# Adjust Image Size from (256, 256, 3) to (224, 224, 3)
# Preprocess the images to meet what is expected by ResNet
# Add GlobalAveragePooling + Classification Head

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

inputs = keras.Input(shape=(256, 256, 3))

x = layers.Lambda(lambda image: tf.image.resize(image, (224,224)))(inputs)
x = keras.applications.resnet.preprocess_input(x)
x = ResNet_base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(5, activation='softmax')(x)


model_TL1 = tf.keras.Model(inputs, outputs)


**Quiz 2:**

1.   Detail the changes that are performed to the images by the method preprocess_input()
2.   How many weights has the feature extraction sub-model?



In [ ]:
model_TL1.summary()

In [ ]:
# Model Compilation

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

model_TL1.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
# Training (Preferably with a GPU)

history = model_TL1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
)

In [ ]:
# Visualization of Results

import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()


Analyze results. By looking at the chart, what do you think is happening?


In [ ]:
# Top layers are well-trained and now we can unfreeze the weights of the base

ResNet_base.trainable = True

# Compile again

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

model_TL1.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
model_TL1.summary()

In [ ]:
# Resume training.
# It will take a while, even with a GPU. That's why we define a low number of training epochs

history = model_TL1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
)

In [ ]:
# Visualization of Results

import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
#### Task 3 ####

# Transfer Learning using anotgher Pretrained Neural Network
# The model Xception will be applied to the Flowers Dataset https://keras.io/api/applications/xception/

# Retrieve the original Xception model
# Complete the neural network (don't forget preprocessing)
# Train

### Code goes here




**Quiz 3**

Present the complete model that was created for the flowers dataset.
This model must rely on the Xception network to perform feature extraction.
